In [1]:
# IPython objects
from IPython.display import display, Markdown

# Data analysis libraries
import numpy as np
np.random.seed(2020)
import pandas as pd
from scipy.stats import norm

# plotting libraries
import matplotlib.pyplot as plt
%matplotlib inline
hist_kwargs = {"edgecolor":"black"}

# others
from glob import glob
from tqdm.autonotebook import tqdm

# analysis functions
from Functions import *

data_path = "../data/"

/home/agostini/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


## Calibrazione

In [ ]:
calibration_files = glob(data_path+"calibration/*")

In [ ]:
def Open_File_Calibration(data_file):
    with open(data_file) as f:
        tot_ev = len(list(f))

    # loop over events and perform analysis
    Ev_list = []
    selected_ev = 0
    with open(data_file) as f:
        with tqdm(total=tot_ev) as pbar:
            for line in f:
                # read event
                ev, evNum, hits = Read_Data(line)
                # select event
                sel, chambers, n_layers = Select_Events_Calibration(ev, hits)
                
                if sel: selected_ev += 1

                # save most important information, in order to plot or reperform the analysis
                # without reading the whole file again
                Ev_list.append(
                    {
                        "Number"    : evNum,
                        "Dataframe" : ev,
                        "Hits"      : hits,
                        "Accepted"  : sel,
                        "Chambers"  : chambers,
                        "Layers"    : n_layers
                    }
                )

                pbar.update()
                
    print("{:35} = {:d}"    .format("Total number of events in the Run", tot_ev))
    print("{:35} = {:d}"    .format("Number of accepted events"        , selected_ev))
    print("{:35} = {:.4f} %".format("Fraction of accepted events"      , selected_ev/tot_ev*100))
    return Ev_list

In [ ]:
def Calibration(Ev_list):
    tot_ev = len(Ev_list)
    selected_ev = 0
    # number of hits per event
    hits_number = []
    # residuals from good events fit
    Ev_residuals = []
    # x position of the best combinations of points per chamber
    X_position = {"up" : [], "down" : []}
    # local fit differences between chambers
    lf_diff = {"slope" : [], "intercept" : []}
    
    print("Performing fit and analysis")
    with tqdm(total=tot_ev) as pbar:
        for event in Ev_list:
            hits_number.append("Hits")
            if event["Accepted"]:
                selected_ev += 1
                ev = event["Dataframe"]
                chambers = event["Chambers"]
                n_layers = event["Layers"]
        
                #Local linear fit
                lf_results = Local_Fit(ev, chambers, n_layers, exclusion_layer=True)
                #Global linear fit
                gf_results = Global_Fit_Calibration(ev, chambers, lf_results)
                Ev_residuals.append(gf_results["residuals"])

                # x coordinates for the points in the optimal combination after local fit per chamber
                X_position["up"]   += [x[1] for x in lf_results[0]["optimal_comb"]]
                X_position["down"] += [x[1] for x in lf_results[1]["optimal_comb"]]
                # local fit differences
                lf_diff["slope"]    .append(lf_results[0]["slope"]    -lf_results[1]["slope"])
                lf_diff["intercept"].append(lf_results[0]["intercept"]-lf_results[1]["intercept"])
            
            pbar.update()
            
    display(Markdown("#### Hit distribution"))
    
    fig, ax = plt.subplots(figsize=(8,5))
    ax.hist(hits_number, **hist_kwargs)
    ax.set_xlabel("number of hits")
            
    display(Markdown("#### Hit position (cell local coordinates)"))
    
    fig, (ax1, ax2) = plt.subplots(figsize=(15,5), ncols=2)

    ax1.set_title("Up")
    ax1.hist(X_position["up"]    , bins=30, **hist_kwargs)
    ax1.set_xlabel("x [mm]")

    ax2.set_title("Down")
    ax2.hist(X_position["down"], bins=30, **hist_kwargs)
    ax2.set_xlabel("x [mm]")

    plt.show()
    
    display(Markdown("#### Difference in slope and intercept between chambers (local fits)"))
    
    fig, (ax1, ax2) = plt.subplots(figsize=(15,5), ncols=2)

    ax1.set_title("Slope")
    ax1.hist(lf_diff["slope"], bins=30, range=(-0.5,0.5), **hist_kwargs)
    ax1.set_xlabel("Slope")

    ax2.set_title("Intercept")
    ax2.hist(lf_diff["intercept"], bins=30, range=(-200,200), **hist_kwargs)
    ax2.set_xlabel("[mm]")
    
    plt.show()
    
    display(Markdown("#### Residuals in the excluded layer"))
    
    Hit_6_layers = 0
    Hit_7_layers = 0
    Hit_8_layers = 0
    Tot_res      = []
    
    for res in Ev_residuals:
        if len(res) == 0:
            Hit_6_layers += 1
        elif len(res) == 1:
            Hit_7_layers += 1
        elif len(res) == 2:
            Hit_8_layers += 1
        else: continue
        Tot_res += res.tolist()

    fig, (ax1, ax2) = plt.subplots(figsize=(15,5), ncols=2)
    ax1.set_title("Residuals of the points in the excluded layers")
    ax1.hist(Tot_res, bins= 30, range=(-20,20), **hist_kwargs)
    ax1.set_xlabel("[mm]")
    
    labels = ['Hit in 8 layers', 'Hit in 7 layers', 'Hit in 6 layers']
    sizes  = [Hit_8_layers/len(Ev_residuals)*100, 
              Hit_7_layers/len(Ev_residuals)*100, 
              Hit_6_layers/len(Ev_residuals)*100]
    patches, _, _ = ax2.pie(sizes, labels=labels, explode = [.05,.05,.05], autopct='%1.1f%%', shadow=True,
                               wedgeprops={"edgecolor":'black', "linewidth":1.25})
    ax2.legend(patches, labels, loc='upper right', fontsize=14)
    ax2.axis('equal')
    ax2.set_title("Percentage of excluded layers in the fit")
    
    plt.show()
            
    return Ev_residuals, X_position, lf_diff

### Run 260

#### Reading data

In [ ]:
Ev_list = Open_File_Calibration(calibration_files[0])

In [ ]:
Ev_residuals, X_position, lf_diff = Calibration(Ev_list)

In [ ]:
Make_Plot(Ev_list[100], calibration=True)

### Run 261

#### Reading data

In [ ]:
Ev_list = Open_File_Calibration(calibration_files[1])

In [ ]:
Ev_residuals, X_position, lf_diff = Calibration(Ev_list)

### Run 262

#### Reading data

In [ ]:
Ev_list = Open_File_Calibration(calibration_files[2])

In [ ]:
Ev_residuals, X_position, lf_diff = Calibration(Ev_list)

### Run 263

#### Reading data

In [ ]:
Ev_list = Open_File_Calibration(calibration_files[3])

In [ ]:
Ev_residuals, X_position, lf_diff = Calibration(Ev_list)

## Physiscs

In [2]:
physics_file = data_path+"physics/Run000331.txt"#"physics/RunMerged.txt"

In [3]:
def Open_File_Physics(data_file):
    with open(data_file) as f:
        tot_ev = len(list(f))

    # loop over events and perform analysis
    Ev_list = []
    selected_ev = 0
    with open(data_file) as f:
        with tqdm(total=tot_ev) as pbar:
            for line in f:
                # read event
                ev, evNum, hits = Read_Data(line)
                # filter by hit position
                ev_left, ev_right, hits_left, hits_right = Points_Filter(ev)
                # select event
                sel_left,  chambers_left,  n_layers_left  = Select_Events_Calibration(ev_left,  hits_left)
                sel_right, chambers_right, n_layers_right = Select_Events_Calibration(ev_right, hits_right)
                
                if sel_left and sel_right: selected_ev += 1

                # save most important information, in order to plot or reperform the analysis
                # without reading the whole file again
                Ev_list.append(
                    {
                        "Number"    : evNum,
                        "Dataframe" : ev,
                        "Hits"      : hits,
                        "Accepted"  : sel_left and sel_right,
                        "Chambers"  : chambers_left+chambers_right,
                        "Layers"    : [n_layers_left, n_layers_right]
                    }
                )

                pbar.update()
                
    print("{:35} = {:d}"    .format("Total number of events in the Run", tot_ev))
    print("{:35} = {:d}"    .format("Number of accepted events"        , selected_ev))
    print("{:35} = {:.4f} %".format("Fraction of accepted events"      , selected_ev/tot_ev*100))
    return Ev_list

In [6]:
def Physics(Ev_list):
    tot_ev = len(Ev_list)
    selected_ev = 0
    # number of hits selected events
    Ev_hits = {"left":[], "right":[]}
    # angular coefficient of fit
    Ev_slope = {"left":[], "right":[]}
    # residuals from good events fit
    Ev_residuals = {"left":[], "right":[]}
    
    print("Performing fit and analysis")
    with tqdm(total=tot_ev) as pbar:
        for event in Ev_list:
            if event["Accepted"]:
                ev = event["Dataframe"]
                layers = event["Layers"]
        
                # filter by hit position
                ev_left, ev_right, hits_left, hits_right = Points_Filter(ev)
                # save hit numbers
                Ev_hits["left"] .append(hits_left )
                Ev_hits["right"].append(hits_right)
                
                # save slope values
                gf_results_left  = Global_Fit_Physics(ev_left , layers[0])
                gf_results_right = Global_Fit_Physics(ev_right, layers[1])
                Ev_slope["left"] .append(gf_results_left ["slope"])
                Ev_slope["right"].append(gf_results_right["slope"])
                
                # save residuals
                Ev_residuals["left"]  += gf_results_left ["residuals"]
                Ev_residuals["right"] += gf_results_right["residuals"]
                
            pbar.update()
            
    display(Markdown("#### Hit distribution (selected events)"))
    
    fig, (ax1, ax2) = plt.subplots(figsize=(15,5), ncols=2)
    ax1.hist(Ev_hits["left"] , title="Left side" , **hist_kwargs)
    ax1.set_xlabel("Number of hits")
    ax2.hist(Ev_hits["right"], title="Right Side", **hist_kwargs)
    ax2.set_xlabel("Number of hits")
    plt.show()
    
    display(Markdown("#### Slope distribution (selected events)"))
    
    fig, (ax1, ax2) = plt.subplots(figsize=(15,5), ncols=2)
    ax1.hist(Ev_slope["left"] , title="Left side" , **hist_kwargs)
    ax1.set_xlabel("Slope")
    ax2.hist(Ev_slope["right"], title="Right Side", **hist_kwargs)
    ax2.set_xlabel("Slope")
    plt.show()
    
    display(Markdown("#### Residuals"))
    
    fig, (ax1, ax2) = plt.subplots(figsize=(15,5), ncols=2)
    ax1.hist(Ev_residuals["left"] , title="Left side" , **hist_kwargs)
    ax1.set_xlabel("Residuals [mm]")
    ax2.hist(Ev_residuals["right"], title="Right Side", **hist_kwargs)
    ax2.set_xlabel("Residuals [mm]")
    plt.show()
            
    return Ev_hits, Ev_slope, Ev_residuals

In [7]:
Ev_list = Open_File_Physics(physics_file)


Total number of events in the Run   = 34428
Number of accepted events           = 121
Fraction of accepted events         = 0.3515 %


In [8]:
Physics(Ev_list)

Performing fit and analysis


NameError: name 'dispaly' is not defined

In [ ]:
idx = 0
for i, ev in enumerate(Ev_list):
    if ev["Accepted"]: 
        idx = i
        break
print(idx)
Make_Plot(Ev_list[idx])